In [131]:
##   注意dataframe 类型数据iloc很慢，可以用apply函数尝试，或者替换成ndarray格式
class DecisionTreeRegression():
    def __init__(self,max_depth: int = None,min_samples_split:int = 5,
         min_samples_leaf: int = 5,min_impurity_decrease: float =0.0):
        '''
        min_samples_split:  内部节点再划分所需最小样本数
        min_samples_leaf:   叶子节点最少样本数 这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝
        分裂需要满足的最小增益
        max_depth: 最大深度
        min_impurity_decrease:分裂需要满足的最小增益
        '''
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_impurity_decrease = min_impurity_decrease
        self.nodes = 0  
        self.tree = None
        self.type_feature = None
        self.index = None
    def __MSE(self,y):
        '''
        :param data: 
        :param y: 目标数据
        :return: MSE: 返回该分支的MSE
        '''
        ##  根据第一个公式
        mean = np.mean(y)  
        mse = np.sum((y-mean)**2)
        return mse

    def __typeFeature(self,X):
        # 表示特征是否为连续还是离散
        n_sample,n_feature = X.shape
        self.type_feature = []
        ####   特征属性小于10个，认为是离散型数据用0表示，连续性数据用1 表示
        for f_idx in range(n_feature):
            if len(np.unique(X[:, f_idx]))< 10:
                self.type_feature.append(0)
            else:
                self.type_feature.append(1)
        return self.type_feature
                

    def __binSplitData(self,X,y,index,f_idx,f_val):
        ### att 数有数据在第f_idx的特征的所有属性,将不等于 f_val 分为一类，其余分为另一类
        ####################    0: 离散类型特征二分方法 1:连续数据   ############################
        att=X[:, f_idx]
        
        if self.type_feature[f_idx]== 0:
            X_left = X[att == f_val]
            X_right = X[att != f_val]
            y_left = y[att == f_val]
            y_right = y[att != f_val]
            index_left = index[att == f_val]
            index_right = index[att != f_val]
        else:
            X_left = X[att <= f_val]
            X_right = X[att >f_val]
            y_left = y[att <= f_val]
            y_right = y[att > f_val]
            index_left = index[att <= f_val]
            index_right = index[att > f_val]
           ## 切分点和样本点的索引
        return X_left, X_right, y_left, y_right,index_left,index_right
    
    
    def __bestSplit(self,X,y,index):
        '''
           
        找到最佳分割特征与特征值
        :param X
        :return: best_f_idx  最佳分割特征 ， best_f_val 特征值
         
        '''
        best_mse = self.__MSE(y)
        n_sample,n_feature = X.shape
        best_f_idx = None
        best_f_val = np.mean(y)
        ## 第一个终止条件： 当叶子节点中的样本数小于最小分割值，不再分割
        if n_sample < self.min_samples_split:
            return best_f_idx,best_f_val       
        ##-------------------------通过不断二分的过程 寻找对于某个特征，的最佳分割点---------------------------
        for f_idx in range(n_feature):
        ##-------------------------如果该特征中的属性个数小于10，则认为是离散数据 type_feature = 0，否则else---------------------------

            if self.type_feature[f_idx] == 0:
                for f_val in np.unique(X[:, f_idx]):
                    ## 当某个特征只有两个类别时，仅仅做一次左右子树的划分，不用重复操作
                    if len(np.unique(X[:, f_idx]))== 2 and f_val == np.unique(X[:, f_idx])[0]:
                        continue

                    else:
                        X_left, X_right, y_left, y_right,index_left,index_right = self.__binSplitData(X,y,index,f_idx,f_val)

                    ## 第二个终止条件： 分割后样本数据小于节点的最低样本数，则放弃分割   
                        if len(index_left)<self.min_samples_leaf or len(index_right)<self.min_samples_leaf:
                            continue
                        mse = self.__MSE(y_left) + self.__MSE(y_right)
                    ## 第三个终止条件，当分裂后的增益小于阈值后者大于目前最大增益
                        if mse < self.min_impurity_decrease or mse > best_mse: 
                            continue
                        else:
                            ## 更新最大增益和最佳分裂位置
                            best_mse = mse
                            best_f_idx,best_f_val = f_idx,f_val
        ##-------------------------     连续特征属性的二分 case = 1   ---------------------------
            else:
                for f_val in np.linspace(X[:, f_idx].min()+1,X[:, f_idx].max()-1,num=50):
                        X_left, X_right, y_left, y_right,index_left,index_right = self.__binSplitData(X,y,index,f_idx,f_val)

                    ## 第二个终止条件： 分割后样本数据小于节点的最低样本数，则放弃分割   
                        if len(index_left)<self.min_samples_leaf or len(index_right)<self.min_samples_leaf:
                            continue
                        mse = self.__MSE(y_left) + self.__MSE(y_right)
                         ## 第三个终止条件，当分裂后的增益小于阈值后者大于目前最大增益
                        if mse < self.min_impurity_decrease or mse > best_mse: 
                            continue
                        else:
                            ## 更新最大增益和最佳分裂位置
                            best_mse = mse
                            best_f_idx,best_f_val = f_idx,f_val
        return best_f_idx,best_f_val

    def __CART(self,X,y,index):
        '''
        生成CART树
        :param X： 特征数据
        :param y: 目标数据
        :return; CART 树
        '''
        best_f_idx, best_f_val = self.__bestSplit(X,y,index)
        self.nodes += 1
        
       
        # best_f_idx 为空表示不能接续划分，则该点为叶子结点  best_f_val
        if best_f_idx is None:
            return index
        # 节点数超过最大深度的限制，也要返回叶节点，叶节点的值为当前数据中的目标值众数
        if self.max_depth:
            if self.nodes >= 2**self.max_depth:
                return index
        tree = dict()
        tree['cut_f'] = best_f_idx
        tree['cut_val'] = best_f_val
        X_left, X_right, y_left, y_right,index_left,index_right = self.__binSplitData(X,y,index,best_f_idx,best_f_val)
        tree['left_value'] = np.mean(y_left)
        tree['right_value'] = np.mean(y_right)
        tree['left'] = self.__CART(X_left,y_left,index_left)
        tree['right'] = self.__CART(X_right,y_right,index_right)
        return tree       
   
    
    def fit(self,X,y,sample_weight = None):
        '''
        拟合模型，数据应该是 ndarray or series类型，dataframe通过 df.values转变成ndarray，不会报错
        :param X: 特征数据
        :param: y: 目标数据
        :param: sample_weight
        :return: None
        '''
        if sample_weight is None:
            ## 使得每个数据的权值都是 1/len(X)    *len(X)是产生 len(X)个
            sample_weight = np.array([1/len(X)] * len(X))
        # 标记每个特征是离散还是连续，从而采用不同的二分方法
        self.index = np.array(range(len(X)))
        self.type_feature = self.__typeFeature(X) 
        self.tree = self.__CART(X,y,self.index)
        return self.tree
    def predict(self,X_test):
        '''
        数据类别预测
        :param X_test:预测数据
        :return: y_: 类别预测结果
        '''

        return np.array([self.__predict_one(x_test, self.tree) for x_test in X_test])
    
    def __predict_one(self,x_test,tree,label = None):
        if isinstance(tree, dict):  # 非叶节点才做左右判断
           
            cut_f_idx, cut_val = tree['cut_f'], tree['cut_val']
            if self.type_feature[cut_f_idx] == 0:
                sub_tree = tree['left'] if x_test[cut_f_idx] == cut_val else tree['right']
                label = tree['left_value'] if x_test[cut_f_idx] == cut_val else tree['right_value']
            else:
                sub_tree = tree['left'] if x_test[cut_f_idx] <= cut_val else tree['right']
                label = tree['left_value'] if x_test[cut_f_idx] <=  cut_val else tree['right_value']
            return self.__predict_one(x_test, sub_tree,label)
        else:
            return label

In [182]:
class GBDTRegression():
    def __init__(self,estimators: int = 10, classifier = DecisionTreeRegression,step: float = 0.1):
        self.estimators = estimators
        self.weakLearner = classifier
        self.step = step
        self.trees = []
        self.F_init = None
        
    def pseudoResiduals(self,y,predicted):
        
        rm = y - predicted
        return rm
    
    def TerminalRegions(self,tree):
        ###  找到每一个叶子节点内的数据，或者说找到叶子节点包含的区域
        global Rm
        for key, val in tree.items():
            if key == 'left' or key =='right':
                if isinstance(tree[key],dict):
                    self.TerminalRegions(tree[key])
                else:
                    Rm.append(val)
        return Rm
    def findRegions(self,x,Rm):
        for i in rangr(len(Rm)):
            (x == Rm[s[1]]).sum()

    def fit(self,X,y):
        self.F_init = np.mean(y)
        ## step1 通过寻找损失函数最小是对应的来设置初值
        F_before = np.array([np.mean(y)] * len(X)) 
        for m in range(self.estimators):
            ##(a) 计算损失函数的负梯度值（也叫伪残差）
            rm = self.pseudoResiduals(y,F_before)
            ## (b) 建立学习器拟合以上残差，
            tree_clf = self.weakLearner(max_depth = 4)
            tree = tree_clf.fit(X, rm)
            self.trees.append(tree_clf)
            ## 并建立每个叶子节点最终区域
            global Rm
            Rm = []
            Rm = self.TerminalRegions(tree)
            Jm = len(Rm)
            gamma_m = np.zeros(len(X)) 
            ## （c）通过最小化每个节点中数据的损失函数和
            for j in range(Jm):
                gamma_m[Rm[j]] += np.mean(rm[Rm[j]])
            ##  (d)更新
            Fm = F_before + self.step *  gamma_m
            F_before = Fm
            
    def predict(self,x_test):
        M = self.estimators
        y_ = np.array([self.F_init] * len(x_test)) 
        for m in range(M):
            a=  self.trees[m].predict(X_test)
           # print('a.shape -----------------------------',a.shape,y_.shape)
            y_ += self.step * self.trees[m].predict(X_test)
        return y_      

In [185]:
if __name__ == '__main__':
    from sklearn import datasets
    import pandas as pd
    import  numpy as np    
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    house_dataset = datasets.load_boston();    #加载波士顿房价数据集
    scaler = StandardScaler()
    X = scaler.fit_transform(house_dataset.data)
    Y = house_dataset.target
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
    tree_clf = GBDTRegression(estimators = 100)
    tree_clf.fit(X_train, Y_train)
    Y_pred = tree_clf.predict(X_test)
    print('acc:{}'.format(np.sum((Y_pred - Y_test)**2) / len(Y_test)))
    del tree_clf
    from sklearn.ensemble import GradientBoostingRegressor 

    tree_clf = GradientBoostingRegressor()

    tree_clf.fit(X_train, Y_train)

    Y_pred = tree_clf.predict(X_test)
    print('sklearn acc:{}'.format(np.sum((Y_pred - Y_test)**2) / len(Y_test)))

acc:9.884889906420687
sklearn acc:8.535771080365048


acc:22.753196266573013


In [180]:
Y_pred

array([17.57891766, 35.35708878, 21.54893373, 14.50878362, 39.64773849,
       13.21248439, 31.53645286, 29.13718817, 19.56760164, 23.28742579,
       26.6419921 , 22.08780486, 24.61433953, 22.00347024, 29.56522168,
       17.51526871, 20.93825286, 24.72178488, 11.1723631 , 20.73303011,
       31.04410102, 16.22635696, 28.86828633, 19.47160032, 16.30586163,
       27.4336788 , 42.25673537, 26.53003248, 27.48087024, 15.03062551,
       31.55963188, 28.42785616, 26.89587815, 14.4454439 , 21.79319128,
       23.38463672, 14.80201895, 20.5337216 , 23.25737652, 18.24862041,
       32.7417942 , 20.8153666 , 26.18889394, 21.45996363, 14.47867669,
        8.85596219, 43.44263105, 19.00668773, 15.82654107, 47.312889  ,
       37.49322115, 13.21882953, 23.2191608 , 23.14380296, 23.59933316,
       18.92533288, 20.10005454, 28.28696423,  7.92440958, 17.86400867,
       26.49493648, 16.70176043, 18.09692757, 17.49296727, 25.23495782,
       21.03396413, 26.51318059, 15.98838545, 16.46983476, 11.26

In [181]:
Y_test

array([20.9, 36.2, 15.6, 13.4, 50. , 11.3, 37. , 24.4, 12.7, 19.7, 23.7,
       22.2, 26.5, 24.4, 36. , 13.9, 19.4, 22. , 12.3, 20.2, 24. , 23.1,
       50. , 19.5, 17.5, 29.9, 46. , 32. , 22.4, 17.1, 28.1, 50. , 25. ,
       14.1, 17.1, 21.4, 14.8, 19.5, 23.2, 19.4, 50. , 18.4, 24.5, 16.6,
       18.4, 11.7, 44.8, 18.3, 11.8, 43.5, 15. , 13.4, 23.2, 24.1, 23.3,
       18.5, 21.7, 36.5,  8.4, 20.6, 24.8, 16.6, 18.7, 17.4, 26.6, 24.5,
       24.5, 11.9, 19.3, 18. , 18.9, 19.3, 26.7, 33.2, 19.9, 16.5, 25. ,
       24.7, 17.8,  8.5,  8.4, 23.8, 14.3, 19.1,  5. , 31. , 11.5, 19.8,
       16.3, 34.9, 41.3, 23.6, 31.7, 10.5, 22. , 15.7, 22.6, 16.1, 36.1,
       19.6, 20.5, 13.1])

In [48]:
c = list(set(b)-set(a))

In [53]:
s = [a,c]

In [54]:
s

[[1, 2], [0, 3, 4, 5]]

In [55]:
s[0]

[1, 2]

In [56]:
s[1]

[0, 3, 4, 5]

In [57]:
b[s[1]]

array([0, 3, 4, 5])

In [58]:
0 == b[s[1]]

array([ True, False, False, False])

In [59]:
(0 == b[s[1]]).sum()

1